In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from tqdm import tqdm_notebook as tqdm
from collections import OrderedDict

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
data = pd.read_csv('../input/hackerearths-snakes-in-the-hood/dataset/train.csv')
print(data.head())
data.breed.value_counts()
#print(data.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(data.breed)
labels = le.transform(data.breed)
data['labels'] = pd.DataFrame(labels)
data.head()

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, data, root_dir):
        self.data = data
        self.root_dir = root_dir
        self.transform = transforms.Compose([transforms.ToPILImage(), 
                                transforms.Resize((256, 256)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.RandomRotation(60),
                                transforms.RandomRotation(20),
#                                 transforms.RandomRotation(120),
#                                 transforms.ColorJitter(),
                                transforms.RandomCrop(256, padding=32),
                                transforms.ToTensor()])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        img_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 0]+'.jpg')
        image = io.imread(img_name)
        image = self.transform(image)
        labels = self.data.iloc[idx, 2]
        labels = np.array([labels])
        #sample = {'images': image, 'labels': labels}

        return (image, labels)

In [ ]:
dataset = CustomDataset(data = data,
                        root_dir='../input/hackerearths-snakes-in-the-hood/dataset/train')

len(data)

In [ ]:
total_count = len(data)

train_count = int(0.8 * total_count) 
test_count = total_count - train_count

train_dataset, test_dataset = torch.utils.data.random_split(dataset, (train_count, test_count))

train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)  
test_dataset_loader  = torch.utils.data.DataLoader(test_dataset , batch_size=64, shuffle=False)
dataloaders = {'train': train_dataset_loader, 'test': test_dataset_loader}

In [ ]:
torch.cuda.is_available()

In [ ]:
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
        
#         self.cnn1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, stride = 1, padding = 1)
#         self.batchnorm1 = nn.BatchNorm2d(8)
#         self.relu = nn.ReLU()
#         self.maxpool = nn.MaxPool2d(kernel_size = 2)
        
#         self.cnn2 = nn.Conv2d(in_channels = 8, out_channels = 32, kernel_size = 5, stride = 1, padding = 2)
#         self.batchnorm2 = nn.BatchNorm2d(32)
        
#         self.cnn3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5, stride = 1, padding = 2)
#         self.batchnorm3 = nn.BatchNorm2d(64)
        
#         self.cnn4 = nn.Conv2d(in_channels = 64, out_channels = 256, kernel_size = 5, stride = 1, padding = 2)
#         self.batchnorm4 = nn.BatchNorm2d(256)
        
#         self.dropout = nn.Dropout(p = 0.7)
#         self.fc1 = nn.Linear(256*256, 2000)
#         self.fc2 = nn.Linear(2000, 1000)
#         self.fc3 = nn.Linear(1000, 350)
        
#     def forward(self, x):
#         out = self.cnn1(x)
#         out = self.batchnorm1(out)
#         out = self.relu(out)
#         out = self.maxpool(out)
        
#         out = self.cnn2(out)
#         out = self.batchnorm2(out)
#         out = self.relu(out)
#         out = self.maxpool(out)
        
#         out = self.cnn3(out)
#         out = self.batchnorm3(out)
#         out = self.relu(out)
#         out = self.maxpool(out)
        
#         out = self.cnn4(out)
#         out = self.batchnorm4(out)
#         out = self.relu(out)
#         out = self.maxpool(out)
        
#         out = out.view(-1, 256*256)
        
#         out = self.fc1(out)
#         out = self.relu(out)
#         out = self.dropout(out)
        
#         out = self.fc2(out)
#         out = self.relu(out)
#         out = self.dropout(out)
        
#         out = self.fc3(out)
#         return out

In [ ]:
from sklearn.metrics import f1_score


def train():
    model.train()
    all_pred = []
    all_labels = []
    tk0 = tqdm(train_dataset_loader, total=int(len(train_dataset_loader)))
    scheduler.step()
    
    for inputs, labels in tk0:
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)

        outputs = model(inputs)
        optimizer.zero_grad()
        loss = criterion(outputs, labels.squeeze())
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        all_pred.append(predicted.tolist())
        all_labels.append(labels.tolist())

    flat_pred = np.array([item for sublist in all_pred for item in sublist])
    flat_labels = np.array([item for sublist in all_labels for item in sublist])
    
    print('For epoch {}/{}, training fscore = {:.3f}'.format(epoch+1, epochs, f1_score(flat_pred, flat_labels, average = 'weighted')))
        
def eval():
    model.eval()
    all_pred = []
    all_labels = []

    tk0 = tqdm(test_dataset_loader, total=int(len(test_dataset_loader)))
    
    for inputs, labels in tk0:
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)
        
        outputs = model(inputs)
#         optimizer.zero_grad()
#         loss = criterion(outputs, labels.squeeze())
#         loss.backward()
#         optimizer.step()

        _, predicted = torch.max(outputs, 1)
        all_pred.append(predicted.tolist())
        all_labels.append(labels.tolist())

    flat_pred = np.array([item for sublist in all_pred for item in sublist])
    flat_labels = np.array([item for sublist in all_labels for item in sublist])

    print('For epoch {}/{}, testing fscore = {:.3f}'.format(epoch+1, epochs, f1_score(flat_pred, flat_labels, average = 'weighted')))
            
def predict():
    
    model.eval()
    all_pred = []
    all_labels = []
    
    for inputs, labels in test_set_loader:
        inputs = inputs.to(device, dtype=torch.float)

        outputs = model(inputs)

        _, predicted = torch.max(outputs, 1)
        all_pred.append(predicted.tolist())

    flat_pred = np.array([item for sublist in all_pred for item in sublist])
    return falt_pred

In [ ]:
import torchvision.models as models
model = models.densenet121(pretrained = True)
num_classes = 350

# for param in model.parameters():
#     param.requires_grad = False 
# num_ftrs = model.fc.in_features
# fc = nn.Sequential(OrderedDict([
#     ('fc1', nn.Linear(num_ftrs,1000)),
#     ('relu', nn.ReLU()),
#     ('fc2', nn.Linear(1000, 350)),
#     ('output', nn.LogSoftmax(dim=1))
# ]))
# model.fc = fc

# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, num_classes)

# model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))

model.classifier = nn.Linear(1024, num_classes)

# model = CNN()

model.to(device, dtype=torch.float)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma = 0.1)


epochs = 35

for epoch in range(epochs):
    train()
    eval()

In [ ]:
test = pd.read_csv('../input/hackerearths-snakes-in-the-hood/dataset/test.csv')
test['labels2'] = 1
test['labels'] = 1
print(test.head())

test_set = CustomDataset(data = test,
                         root_dir='../input/hackerearths-snakes-in-the-hood/dataset/test')

test_set_loader  = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

In [ ]:
all_pred = []
all_labels = []

for inputs, labels in test_set_loader:
    inputs = inputs.to(device, dtype=torch.float)

    outputs = model(inputs)

    _, predicted = torch.max(outputs, 1)
    all_pred.append(predicted.tolist())

flat_pred = np.array([item for sublist in all_pred for item in sublist])

In [ ]:
ans=pd.DataFrame((zip(test['image_id'], flat_pred)), columns=['image_id','breed'])

In [ ]:
ans['breed'] = le.inverse_transform(ans['breed'])

In [ ]:
ans.to_csv('output.csv',index=False)